<a href="https://colab.research.google.com/github/harshitadd/AI4BharatTranslation/blob/main/urlPDF2TXT_with_sentence_splitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!apt-get install poppler-utils
!pip install indic-nlp-library
!pip install pytesseract
!pip install sentence-splitter
!pip install pdf2image
!pip install sentence_transformers

# UPLOAD TRAINDATA FILE # at /usr/share/tesseract-ocr/4.00/tessdata/tam.traineddata

In [8]:
import requests
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os
import sys
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import sentence_tokenize 
from sentence_transformers import SentenceTransformer
import numpy as np
import re 
from sentence_splitter import SentenceSplitter, split_text_into_sentences
from scipy.spatial import distance
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

In [9]:
# Data Source 
from google.colab import drive
drive.mount('/content/gdrive') # CHANGE THIS 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
def source_links():
  with open('/content/gdrive/MyDrive/source_links.txt','r') as file: 
    links = file.read()                                              # l [0] ==> eng link and l[1] ==> tam link 
    links = links.split()
    source_links = []
    for link in links:
      if '_e_' in link:
        l = []
        l.append(link)
        temp = link 
        temp = temp.replace('_e_','_t_')
        l.append(temp)
        source_links.append(l)
    print('Number of Source Links :' + str(len(source_links)))
  return source_links

In [11]:

batched_links = []
# Making Mini Batches for download+preprocess
def minibatch(source_links):
  for j in range(0, len(source_links),20):
    batched_links.append(source_links[j:j+20])
  return batched_links 


In [12]:
def download_pdf_to_image(prim_lid, sec_lid, source_links, batch_index):
  source_counter = 1
  failed = []
  for source in source_links:
    eng = source[0]
    tam = source[1]
    response_eng = requests.get(eng)
    response_tam = requests.get(tam) 
    en = '/content/gdrive/MyDrive/eng' + '_' + str(batch_index) + '_' + str(source_counter) + '.pdf'
    ta = '/content/gdrive/MyDrive/tam' + '_' + str(batch_index) + '_' + str(source_counter) + '.pdf'
    try:
      with open(en, 'wb') as e, open(ta, 'wb') as t:
          e.write(response_eng.content)
          t.write(response_tam.content)   
          pages = convert_from_path(en, 200) 
          image_counter = 1
          for page in pages: 
            filename = "page_"+str(image_counter)+".jpg"
            page.save(filename, 'JPEG') 
            image_counter = image_counter + 1
          filelimit = image_counter-1
          print('English version of ' + str(source_counter) + ' PDF Succesfully Saved as Images')
          outfile = '/content/gdrive/MyDrive/eng' + '_' + str(batch_index) + '_' + str(source_counter) + '.txt' 
          f = open(outfile, "a") 
          content = ''
          for i in range(1, filelimit + 1):
            filename = "page_"+str(i)+".jpg" 
            try: 
              content += str(((pytesseract.image_to_string(Image.open(filename),lang='eng+tam'))))  
            except:
              content +=' '
          content = content.replace('\n',' ')
          splitter = SentenceSplitter(language='en')
          so_sentences = splitter.split(content)
          text = ''
          for s in so_sentences:
            text += s
            text += '\n'
          f.write(text)
          f.close() 
          print(' English PDF Processed Completely')


          # Working on Tamil   
          pages = convert_from_path(ta, 200) 
          image_counter = 1
          skipped = 0
          for page in pages: 
            filename = "page_"+str(image_counter)+".jpg"
            page.save(filename, 'JPEG') 
            image_counter = image_counter + 1
          filelimit = image_counter-1
          print('Tamil version of ' + str(source_counter) + ' PDF Succesfully Saved as Images')

          outfile = '/content/gdrive/MyDrive/tam' + '_' + str(batch_index) + '_' + str(source_counter) + '.txt' 
          f = open(outfile, "a") 
          content = ''
          for i in range(1, filelimit + 1):
            filename = "page_"+str(i)+".jpg" 
            try: 
              content += str(((pytesseract.image_to_string(Image.open(filename),lang='tam+eng'))))  
            except:
              content +=' '
          content = content.replace('\n',' ')
          ta_sentences = sentence_tokenize.sentence_split(content, lang='ta') 
          text = ''
          for s in ta_sentences:
            text += s
            text += '\n'
          f.write(text)
          f.close() 
          print('Tamil PDF Processed Completely')
          source_counter+=1
    except: 
          failed.append((eng,tam))   
          print('Failed for ' + str(source_counter) + 'PDF' )   
    
  print('Batch Complete!')
  print('Total Number of Documents was ' + str(len(source_links)))
  print('URL fetch failed for :' + str(len(failed)))
  

In [13]:
# CHANGE THIS
source_links = source_links() 
prim_lid = 'tam' # Primary lang
sec_lid = 'eng' # Depending upon which language is codeswitched.
batch_index = 6 # Change for different batches 
# # # 

batched_links = minibatch(source_links)
download_pdf_to_image(prim_lid, sec_lid, batched_links[batch_index], batch_index) # Downloading PDFs and storing them on Gdrive
print('Post Processing Complete for : ' + str(batch_index) + ' Batch Number')


Number of Source Links :431
English version of 1 PDF Succesfully Saved as Images
 English PDF Processed Completely
Tamil version of 1 PDF Succesfully Saved as Images
Tamil PDF Processed Completely
English version of 2 PDF Succesfully Saved as Images
 English PDF Processed Completely
Tamil version of 2 PDF Succesfully Saved as Images
Tamil PDF Processed Completely
English version of 3 PDF Succesfully Saved as Images
 English PDF Processed Completely
Tamil version of 3 PDF Succesfully Saved as Images
Tamil PDF Processed Completely
English version of 4 PDF Succesfully Saved as Images
 English PDF Processed Completely
Tamil version of 4 PDF Succesfully Saved as Images
Tamil PDF Processed Completely
English version of 5 PDF Succesfully Saved as Images
 English PDF Processed Completely
Tamil version of 5 PDF Succesfully Saved as Images
Tamil PDF Processed Completely
English version of 6 PDF Succesfully Saved as Images
 English PDF Processed Completely
Tamil version of 6 PDF Succesfully Saved